# Date Histogram Aggregations

Aggregations are one of the most important application of Elasticsearch. It provides you with quick powerful analysis of your data! Below we have discussed aggregations over date values.

A lot of analysis happen on a time-series scales. For example: Quaterly sales of iphone across the world. Therefore it is essential to have an fast aggregation done over large dataset under different granular scales. ES provides such an aggregation via date histogram aggregation. The granularities over which you can do aggregations are:

1. year
2. quater
3. month
4. hour
5. week
6. day
7. hour
8. minute
9. second
10. milisecond

For more info : https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-datehistogram-aggregation.html

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch('127.0.0.1:9200')

In [21]:
doc1 = {"city":"Bangalore", "country":"India","datetime":"2018,01,01,10,20,00"} #datetime format: yyyy,MM,dd,hh,mm,ss
doc2 = {"city":"London", "country":"England","datetime":"2018,01,02,03,12,00"}
doc3 = {"city":"Los Angeles", "country":"USA","datetime":"2018,04,19,05,02,00"}

In [32]:
es.index(index="travel", doc_type="cities", id=1, body=doc1)
es.index(index="travel", doc_type="cities", id=2, body=doc2)
es.index(index="travel", doc_type="cities", id=3, body=doc3)

{u'_id': u'3',
 u'_index': u'travel',
 u'_primary_term': 2,
 u'_seq_no': 2,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'cities',
 u'_version': 3,
 u'result': u'updated'}

In [33]:
es.indices.get_mapping(index='travel', doc_type='cities') #POSTMAN: http://127.0.0.1:9200/travel/_mapping/cities

{u'travel': {u'mappings': {u'cities': {u'properties': {u'city': {u'fields': {u'keyword': {u'ignore_above': 256,
        u'type': u'keyword'}},
      u'type': u'text'},
     u'country': {u'fields': {u'keyword': {u'ignore_above': 256,
        u'type': u'keyword'}},
      u'type': u'text'},
     u'datetime': {u'format': u'yyyy,MM,dd,hh,mm,ss', u'type': u'date'}}}}}}

In [34]:
es.search(index="travel",
              body={"from": 0, "size": 5, "query": {"match_all": {}}})

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'2',
    u'_index': u'travel',
    u'_score': 1.0,
    u'_source': {u'city': u'London',
     u'country': u'England',
     u'datetime': u'2018,01,02,03,12,00'},
    u'_type': u'cities'},
   {u'_id': u'1',
    u'_index': u'travel',
    u'_score': 1.0,
    u'_source': {u'city': u'Bangalore',
     u'country': u'India',
     u'datetime': u'2018,01,01,10,20,00'},
    u'_type': u'cities'},
   {u'_id': u'3',
    u'_index': u'travel',
    u'_score': 1.0,
    u'_source': {u'city': u'Los Angeles',
     u'country': u'USA',
     u'datetime': u'2018,04,19,05,02,00'},
    u'_type': u'cities'}],
  u'max_score': 1.0,
  u'total': 3},
 u'timed_out': False,
 u'took': 2}

In [35]:
es.search(index="travel",
              body={"from": 0, "size": 0, "query": {"match_all": {}}, "aggs": {
                  "country": {
                      "date_histogram": {"field": "datetime", "interval": "year"}}}})

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'country': {u'buckets': [{u'doc_count': 3,
     u'key': 1514764800000,
     u'key_as_string': u'2018,01,01,12,00,00'}]}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 3},
 u'timed_out': False,
 u'took': 5}

In [36]:
doc4 = {"city":"Sydney", "country":"Australia","datetime":"2019,01,01,10,20,00"}
es.index(index="travel", doc_type="cities", id=4, body=doc4)

{u'_id': u'4',
 u'_index': u'travel',
 u'_primary_term': 2,
 u'_seq_no': 4,
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'cities',
 u'_version': 1,
 u'result': u'created'}

In [41]:
es.search(index="travel",
              body={"from": 0, "size": 0, "query": {"match_all": {}}, "aggs": {
                  "country": {
                      "date_histogram": {"field": "datetime", "interval": "year"}}}})

{u'_shards': {u'failed': 0, u'skipped': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'country': {u'buckets': [{u'doc_count': 3,
     u'key': 1514764800000,
     u'key_as_string': u'2018,01,01,12,00,00'},
    {u'doc_count': 1,
     u'key': 1546300800000,
     u'key_as_string': u'2019,01,01,12,00,00'}]}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 4},
 u'timed_out': False,
 u'took': 2}